In [23]:
#Importando bibliotecas

import numpy as np

import torch
import torch.nn as nn
    
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader

from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch.nn as nn
import plotly.graph_objects as go

# CPC SSL Training

In [24]:
# Dados de Treino

data_path = Path('/workspaces/betania/ssl_tools/view_concatenated/WISDM_cpc/train')

datas_x_train = []

data_y_train = []

for f in data_path.glob('*.csv'):
    data = pd.read_csv(f)
    x = data[['accel-x', 'accel-y', 'accel-z', 'gyro-x', 'gyro-y', 'gyro-z']].values

    # Expend dimension

    x = np.swapaxes(x, 1, 0)
    
    datas_x_train.append(x)

    y = data['standard activity code'].values

    data_y_train.append(y)


In [25]:
# Use a função shape para obter o shape de cada array na lista
formas = [arr.shape for arr in datas_x_train]

# Isso irá imprimir as formas (shapes) de cada array na lista
for i, forma in enumerate(formas):
    print(f"Array {i+1}: Shape {forma}")

Array 1: Shape (6, 17969)
Array 2: Shape (6, 17974)
Array 3: Shape (6, 17994)
Array 4: Shape (6, 17965)
Array 5: Shape (6, 17991)
Array 6: Shape (6, 17964)
Array 7: Shape (6, 17982)
Array 8: Shape (6, 35935)
Array 9: Shape (6, 17964)
Array 10: Shape (6, 17983)
Array 11: Shape (6, 17964)
Array 12: Shape (6, 17967)
Array 13: Shape (6, 17994)
Array 14: Shape (6, 17966)
Array 15: Shape (6, 17982)
Array 16: Shape (6, 17991)
Array 17: Shape (6, 17966)
Array 18: Shape (6, 17966)
Array 19: Shape (6, 17965)
Array 20: Shape (6, 17963)
Array 21: Shape (6, 17967)
Array 22: Shape (6, 17983)
Array 23: Shape (6, 17964)
Array 24: Shape (6, 17972)
Array 25: Shape (6, 17984)
Array 26: Shape (6, 17964)
Array 27: Shape (6, 17967)
Array 28: Shape (6, 17965)
Array 29: Shape (6, 17995)
Array 30: Shape (6, 17963)
Array 31: Shape (6, 17982)
Array 32: Shape (6, 17985)
Array 33: Shape (6, 17981)
Array 34: Shape (6, 17970)
Array 35: Shape (6, 17968)
Array 36: Shape (6, 17965)


In [26]:
# Dados de Teste

data_path = Path('/workspaces/betania/ssl_tools/view_concatenated/WISDM_cpc/test')

datas_x_test = []

data_y_test = []

for f in data_path.glob('*.csv'):
    data = pd.read_csv(f)
    x = data[['accel-x', 'accel-y', 'accel-z', 'gyro-x', 'gyro-y', 'gyro-z']].values

    # Expend dimension

    x = np.swapaxes(x, 1, 0)
    
    datas_x_test.append(x)

    y = data['standard activity code'].values

    data_y_test.append(y)

In [27]:
# y = data['standard activity code'].values
# y

In [28]:
x_train = datas_x_train
y_train = data_y_train
x_test = datas_x_test
y_test = data_y_test

In [29]:
y_test

[array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1]),
 array([2, 2, 2, ..., 1, 1, 1])]

In [30]:
# Resume todos os vetores z em em único vetor de contexto​

class GRUEncoder(torch.nn.Module):
    def __init__(
        self,
        hidden_size: int = 100,
        in_channel: int = 6,
        encoding_size: int = 10,
        num_layers: int = 1,
        dropout: float = 0.0,
        bidirectional: bool = True,
        device: str = "cpu",
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.in_channel = in_channel
        self.num_layers = num_layers
        self.encoding_size = encoding_size
        self.bidirectional = bidirectional
        self.device = device
        
        self.rnn = torch.nn.GRU(
            input_size=self.in_channel,
            hidden_size=self.hidden_size,
            num_layers=num_layers,
            batch_first=False,
            dropout=dropout,
            bidirectional=bidirectional,
        ).to(device) 

        self.nn = torch.nn.Linear(
            self.hidden_size * (int(self.bidirectional) + 1), self.encoding_size
        ).to(device)

    def forward(self, x):
        x = x.permute(2, 0, 1)

        past = torch.zeros(
            self.num_layers * (int(self.bidirectional) + 1),
            x.shape[1],
            self.hidden_size,
            device=self.device,
        )

        out, _ = self.rnn(
            x, past
        )  # out shape = [seq_len, batch_size, num_directions*hidden_size]
        encodings = self.nn(out[-1].squeeze(0))
        return encodings

In [31]:
class CPC(pl.LightningModule):
    def __init__(
        self,
        encoder: torch.nn.Module,
        density_estimator: torch.nn.Module,
        auto_regressor: torch.nn.Module,
        lr: float = 1e-3,
        weight_decay: float = 0.0,
        window_size: int = 4,
        n_size: int = 5
    ):
        super().__init__()
        self.encoder = encoder.to(self.device)
        self.density_estimator = density_estimator.to(self.device)
        self.auto_regressor = auto_regressor.to(self.device)
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.window_size = window_size
        self.n_size = n_size
        self.training_step_losses = []


    def training_step(self, batch, batch_idx):
        assert len(batch) == 1, "Batch must be 1 sample only"
        sample = batch
        sample = sample.squeeze(0)
        rnd_t = np.random.randint(
            5 * self.window_size, sample.shape[-1] - 5 * self.window_size
        )
        sample = torch.tensor(
            sample[
                :,
                max(0, (rnd_t - 20 * self.window_size)) : min(
                    sample.shape[-1], rnd_t + 20 * self.window_size
                ),
            ]
        ).cpu()

        T = sample.shape[-1]
        windowed_sample = np.split(
            sample[:, : (T // self.window_size) * self.window_size],
            (T // self.window_size),
            -1,
        )
        windowed_sample = torch.tensor(np.stack(windowed_sample, 0), device=self.device)
        encodings = self.encoder(windowed_sample)
        window_ind = torch.randint(2, len(encodings) - 2, size=(1,))
        _, c_t = self.auto_regressor(
            encodings[max(0, window_ind[0] - 10) : window_ind[0] + 1].unsqueeze(0)
        )
        density_ratios = torch.bmm(
            encodings.unsqueeze(1),
            self.density_estimator(c_t.squeeze(1).squeeze(0)).expand_as(encodings).unsqueeze(-1),
        ).view(
            -1,
        )
        r = set(range(0, window_ind[0] - 2))
        r.update(set(range(window_ind[0] + 3, len(encodings))))
        rnd_n = np.random.choice(list(r), self.n_size)
        X_N = torch.cat(
            [density_ratios[rnd_n], density_ratios[window_ind[0] + 1].unsqueeze(0)], 0
        )
        labels = torch.Tensor([len(X_N) - 1]).to(self.device)
        loss = torch.nn.CrossEntropyLoss()(X_N.view(1, -1), labels.long())
        self.training_step_losses.append(loss)
        return loss


    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.training_step_losses.clear()
        
    def configure_optimizers(self):
        learnable_parameters = (
            list(self.density_estimator.parameters())
            + list(self.encoder.parameters())
            + list(self.auto_regressor.parameters())
        )

        optimizer = torch.optim.Adam(
            learnable_parameters, lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer

In [32]:
encoding_size = 256
encoder = GRUEncoder(encoding_size=256, device='cuda')
density_estimator = torch.nn.Linear(encoding_size, encoding_size)
auto_regressor = torch.nn.GRU(
    input_size=encoding_size, 
    hidden_size=encoding_size, 
    batch_first=True
)

cpc = CPC(encoder, density_estimator, auto_regressor)

In [33]:
class SimpleDataset:
    def __init__(self, X, y=None):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
        else:
            return self.X[idx].astype(np.float32)
    
train_dataset = SimpleDataset(x_train)
test_dataset = SimpleDataset(x_test, y_test)
len(train_dataset), len(test_dataset)

(36, 11)

In [34]:
train_loader = DataLoader(train_dataset, batch_size=1, num_workers=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=10, shuffle=False)

In [35]:
trainer = pl.Trainer(max_epochs=500, accelerator="gpu", devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [36]:
trainer.fit(cpc, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name              | Type       | Params
-------------------------------------------------
0 | encoder           | GRUEncoder | 116 K 
1 | density_estimator | Linear     | 65.8 K
2 | auto_regressor    | GRU        | 394 K 
-------------------------------------------------
576 K     Trainable params
0         Non-trainable params
576 K     Total params
2.307     Total estimated model params size (MB)
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (36) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:  69%|██████▉   | 25/36 [00:00<00:00, 126.30it/s, v_num=136]

/tmp/ipykernel_280400/3311096431.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(


Epoch 1:   0%|          | 0/36 [00:00<?, ?it/s, v_num=136]          

Epoch 499: 100%|██████████| 36/36 [00:00<00:00, 50.96it/s, v_num=136, train_loss=1.740]

`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|██████████| 36/36 [00:00<00:00, 42.40it/s, v_num=136, train_loss=1.740]


## CPC Fine-Tuning

We are going to fine-tune the CPC model on the downstream task of classification. We will use the same dataset and re-use the same encoder.

In [37]:
from typing import Any, Optional

from torchmetrics.functional import accuracy


class StateClassifier(torch.nn.Module):
    def __init__(self, input_size: int = 10, n_classes: int = 7):
        super(StateClassifier, self).__init__()
        self.input_size = input_size
        self.n_classes = 7
        self.normalize = torch.nn.BatchNorm1d(self.input_size)
        self.nn = torch.nn.Linear(self.input_size, self.n_classes)
        torch.nn.init.xavier_uniform_(self.nn.weight)

    def forward(self, x):
        x = self.normalize(x)
        logits = self.nn(x)
        return logits


class CPC_Classifier(pl.LightningModule):
    def __init__(
        self,
        encoder: torch.nn.Module,
        classifier: torch.nn.Module,
        lr: float = 1e-3,
        weight_decay: float = 0.0,
        task_class: str = "multiclass",
        num_classes: int = 7
    ):
        super().__init__()
        self.encoder = encoder.to(self.device)
        self.classifier = classifier.to(self.device)
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.training_step_losses = []
        self.validation_step_losses = []
        self.loss_function = torch.nn.CrossEntropyLoss()
        self.task_class = task_class
        self.num_classes = 7
        
    def configure_optimizers(self) -> Any:
        optimizer = torch.optim.Adam(
            self.classifier.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer
    
    def forward(self, x):
        encodings = self.encoder(x)
        predictions = self.classifier(encodings)
        return predictions
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        self.training_step_losses.append(loss)
        return loss
    
    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.training_step_losses.clear()
    
    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        self.validation_step_losses.append(loss)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics)
        return metrics
    
    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.log_dict(metrics)
        return metrics

        
    def on_validation_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.validation_step_losses).mean()
        self.log("val_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.validation_step_losses.clear()

    def _shared_eval_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        acc = accuracy(torch.argmax(predictions, dim=1), y.long(), task=self.task_class, num_classes=self.num_classes)
        return loss, acc

In [38]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 100

# Caminho dos dados

data_path_train = Path('/workspaces/betania/data/standartized_balanced/WISDM/train.csv')

data_path_validation = Path('/workspaces/betania/data/standartized_balanced/WISDM/validation.csv')

data_path_test = Path('/workspaces/betania/data/standartized_balanced/WISDM/test.csv')

# Train

x_train = pd.read_csv(data_path_train)

x_train = x_train.iloc[:, :360]
    
x_train = x_train.astype(np.float32)

y_train = pd.read_csv(data_path_train)

y_train = y_train.iloc[:, -1]

y_train = y_train.astype(np.float32)

tensor_x = torch.Tensor(np.array(x_train))

tensor_y = torch.Tensor(np.array(y_train))

original_dim = tensor_x.shape[0]

input_shape = (original_dim, 6, 60)

tensor_x = tensor_x.reshape(input_shape)

dataset_train= TensorDataset(tensor_x,tensor_y)

dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, drop_last= True)

# Validation

x_validation = pd.read_csv(data_path_validation)

x_validation = x_validation.iloc[:, :360]

x_validation = x_validation.astype(np.float32)

y_validation = pd.read_csv(data_path_validation)

y_validation = y_validation.iloc[:, -1]

y_validation = y_validation.astype(np.float32)

tensor_x_val = torch.Tensor(np.array(x_validation))

tensor_y_val = torch.Tensor(np.array(y_validation))

original_dim_val = tensor_x_val.shape[0]

tensor_x_val = tensor_x_val.reshape(original_dim_val, 6, 60)

dataset_val= TensorDataset(tensor_x_val,tensor_y_val)

dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True, drop_last = True)

# Test

x_test = pd.read_csv(data_path_test)

x_test = x_test.iloc[:, :360]

x_test = x_test.astype(np.float32)

y_test = pd.read_csv(data_path_test)

y_test = y_test.iloc[:, -1]

y_test = y_test.astype(np.float32)

tensor_x_test = torch.Tensor(np.array(x_test))

tensor_y_test = torch.Tensor(np.array(y_test))

original_dim_test = tensor_x_test.shape[0]

tensor_x_test = tensor_x_test.reshape((original_dim_test, 6, 60))

dataset_test= TensorDataset(tensor_x_test,tensor_y_test)

dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True, drop_last= True)


In [39]:
data_path_train = Path('/workspaces/betania/data/standartized_balanced/WISDM/train.csv')

y_train = pd.read_csv(data_path_train)

y_train = y_train.iloc[:, -1]

y_train.unique()

array([2, 5, 6, 0, 1])

In [40]:
x_train

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-50,gyro-z-51,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59
0,-4.406084,-7.971005,-0.280978,8.237112,2.229308,-1.877797,-3.078670,-6.051075,-3.491006,-1.464815,...,-0.776451,-0.576500,-0.575899,-0.799183,-0.084052,0.752021,1.587773,2.650263,1.906534,0.092092
1,0.186370,-0.050579,-0.040649,-0.449850,-0.655638,1.033459,0.007363,-3.121700,2.422174,-2.430532,...,-0.143452,-0.646708,0.207145,0.702881,0.010870,0.190366,-0.486779,-0.079197,0.471224,-0.197064
2,5.524070,0.704957,-7.538637,-8.366249,-0.896879,-2.866145,4.361573,3.237100,3.697426,3.434555,...,-1.438400,-1.782263,-1.537537,-1.536583,-1.961644,-1.557289,0.286954,4.903331,5.336706,3.811437
3,-1.417248,-0.462334,-1.679691,-1.038921,-1.575214,0.749882,4.269343,3.831115,0.655725,-1.144818,...,0.189591,0.444646,0.451423,0.160605,-0.055797,-0.008116,0.130876,0.558405,0.738815,0.314354
4,-6.675334,-4.063461,-3.253331,5.513613,8.510528,8.789780,9.061282,2.349534,-10.563099,-8.488220,...,-1.874475,0.041224,0.816235,0.905279,0.424946,-1.133572,-1.697517,-1.766260,-2.217915,-2.309474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10915,0.035006,0.073410,-0.033312,-0.036785,0.077832,-0.008284,-0.019827,-0.037937,-0.048616,-0.003029,...,0.007244,0.002286,-0.004577,-0.002461,-0.002443,-0.001429,0.003391,-0.000314,0.004041,0.006878
10916,0.130879,0.216577,0.271304,0.435954,0.347626,-0.456578,-0.295231,-0.161805,-1.112699,-0.444722,...,0.007144,0.007289,-0.001360,-0.006746,0.002004,0.000193,0.000345,-0.003875,-0.004737,-0.000545
10917,0.023335,0.013198,-0.008005,-0.012395,0.008797,0.005418,0.006365,0.030876,0.032646,0.032760,...,-0.015202,-0.002399,0.012727,0.028815,0.037020,0.045893,0.046907,0.045081,0.030555,-0.007662
10918,0.034519,-0.001241,-0.061202,-0.021307,0.023097,0.009323,0.012766,0.045946,-0.063485,-0.030942,...,-0.001505,0.000898,0.000791,-0.013288,-0.003919,-0.007930,-0.002467,-0.001422,-0.004019,-0.002352


In [41]:
# shape dataloader

for i, (x, y) in enumerate(dataloader_train):
    print(f"Batch {i} shape: {x.shape}")

for i, (x, y) in enumerate(dataloader_val):
    print(f"Batch {i} shape: {y.shape}")
    

Batch 0 shape: torch.Size([100, 6, 60])
Batch 1 shape: torch.Size([100, 6, 60])
Batch 2 shape: torch.Size([100, 6, 60])
Batch 3 shape: torch.Size([100, 6, 60])
Batch 4 shape: torch.Size([100, 6, 60])
Batch 5 shape: torch.Size([100, 6, 60])
Batch 6 shape: torch.Size([100, 6, 60])
Batch 7 shape: torch.Size([100, 6, 60])
Batch 8 shape: torch.Size([100, 6, 60])
Batch 9 shape: torch.Size([100, 6, 60])
Batch 10 shape: torch.Size([100, 6, 60])
Batch 11 shape: torch.Size([100, 6, 60])
Batch 12 shape: torch.Size([100, 6, 60])
Batch 13 shape: torch.Size([100, 6, 60])
Batch 14 shape: torch.Size([100, 6, 60])
Batch 15 shape: torch.Size([100, 6, 60])
Batch 16 shape: torch.Size([100, 6, 60])
Batch 17 shape: torch.Size([100, 6, 60])
Batch 18 shape: torch.Size([100, 6, 60])
Batch 19 shape: torch.Size([100, 6, 60])
Batch 20 shape: torch.Size([100, 6, 60])
Batch 21 shape: torch.Size([100, 6, 60])
Batch 22 shape: torch.Size([100, 6, 60])
Batch 23 shape: torch.Size([100, 6, 60])
Batch 24 shape: torch.Size

In [42]:
encoding_size = 256
n_classes = 7

classifier = StateClassifier(input_size=encoding_size, n_classes=n_classes)
cpc_classifier = CPC_Classifier(encoder, classifier)

In [43]:
trainer = pl.Trainer(max_epochs=1000, accelerator="gpu", devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [44]:
trainer.fit(cpc_classifier, train_dataloaders=dataloader_train, val_dataloaders=dataloader_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type             | Params
---------------------------------------------------
0 | encoder       | GRUEncoder       | 116 K 
1 | classifier    | StateClassifier  | 2.3 K 
2 | loss_function | CrossEntropyLoss | 0     
---------------------------------------------------
118 K     Trainable params
0         Non-trainable params
118 K     Total params
0.474     Total estimated model params size (MB)


/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Epoch 0:  14%|█▍        | 15/109 [00:00<00:00, 96.64it/s, v_num=137]

Epoch 999: 100%|██████████| 109/109 [00:01<00:00, 65.59it/s, v_num=137, val_loss=0.648, train_loss=0.727] 

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 109/109 [00:01<00:00, 62.77it/s, v_num=137, val_loss=0.648, train_loss=0.727]


## Classification evaluation

In [45]:
trainer.test(cpc_classifier, dataloader_test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 32/32 [00:00<00:00, 132.84it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6371874809265137
        test_loss           0.7074582576751709
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.6371874809265137, 'test_loss': 0.7074582576751709}]